# Fine tune Chinese bert with sentences

CS685 Spring 2022 <br />
Apr. 26, 2022<br />
Hongyu Tu <br />

In [1]:
import torch
import pickle
import datasets 
import pandas as pd
from utils import *
import matplotlib.pyplot as plt

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### Load data

In [3]:
# name = '../data/danmu'

# df = pd.read_csv('{}.csv'.format(name))
# text_raw, label_raw = df['danmu'].to_list(), df['Category ID'].to_list()

# cat_dic, tmp = init_category_dic(), {}
# for i in list(cat_dic.keys()):
#     _, __, main_id = cat_dic[i]
#     tmp[i] = main_id
    
# t_l = list(set(tmp.values()))

# text, label = [], []
# for i in range(len(text_raw)):
#     if label_raw[i] in tmp.keys() \
#         and 'str' in str(type(text_raw[i])) \
#         and len(text_raw[i]) < 25:
#         text.append(text_raw[i])
#         l = t_l.index(tmp[label_raw[i]])
#         label.append(l)
        

# split_idx = int(len(label) * 0.9)
# dataset = datasets.DatasetDict({"train": datasets.Dataset.from_dict({"text": text[:split_idx], "label": label[:split_idx]}), \
#                                 "test": datasets.Dataset.from_dict({"text": text[split_idx:], "label": label[split_idx:]})})

# with open('danmu_chn_bert_full.pkl', 'wb') as f:
#     pickle.dump(dataset, f, protocol=pickle.HIGHEST_PROTOCOL)

In [4]:
with open('danmu_chn_bert_full.pkl', 'rb') as f:
    dataset = pickle.load(f)

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 817000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 90778
    })
})

In [6]:
num_class = np.max(list(set(dataset['test']['label'] + dataset['train']['label'])))
num_class += 1
num_class

21

### Model tuning

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

  0%|          | 0/817 [00:00<?, ?ba/s]

  0%|          | 0/91 [00:00<?, ?ba/s]

In [8]:
# small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
# small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

small_train_dataset = tokenized_datasets["train"].shuffle(seed=42)
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42)

In [9]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-chinese", num_labels=num_class)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [10]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [11]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

In [12]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [13]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", num_train_epochs = 10)

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
C:\Users\tomtu\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 817000
  Num Epochs = 15
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1531875


Epoch,Training Loss,Validation Loss


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\tomtu\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\tomtu\AppData\Local\Temp/ipykernel_7312/4032920361.py", line 1, in <module>
    trainer.train()
  File "C:\Users\tomtu\anaconda3\lib\site-packages\transformers\trainer.py", line 1400, in train
    tr_loss_step = self.training_step(model, inputs)
  File "C:\Users\tomtu\anaconda3\lib\site-packages\transformers\trainer.py", line 2002, in training_step
    loss.backward()
  File "C:\Users\tomtu\anaconda3\lib\site-packages\torch\_tensor.py", line 363, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph, inputs=inputs)
  File "C:\Users\tomtu\anaconda3\lib\site-packages\torch\autograd\__init__.py", line 173, in backward
    Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
KeyboardInterrupt

During handling

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



  File "C:\Users\tomtu\anaconda3\lib\site-packages\IPython\core\ultratb.py", line 281, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "C:\Users\tomtu\anaconda3\lib\inspect.py", line 1541, in getinnerframes
    frameinfo = (tb.tb_frame,) + getframeinfo(tb, context)
  File "C:\Users\tomtu\anaconda3\lib\inspect.py", line 1499, in getframeinfo
    filename = getsourcefile(frame) or getfile(frame)
  File "C:\Users\tomtu\anaconda3\lib\inspect.py", line 709, in getsourcefile
    if getattr(getmodule(object, filename), '__loader__', None) is not None:
  File "C:\Users\tomtu\anaconda3\lib\inspect.py", line 755, in getmodule
    os.path.realpath(f)] = module.__name__
  File "C:\Users\tomtu\anaconda3\lib\ntpath.py", line 664, in realpath
    if _getfinalpathname(spath) == path:
KeyboardInterrupt
Traceback (most recent call last):
  File "C:\Users\tomtu\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3444, in run_